<a href="https://colab.research.google.com/github/Manusua/03MIA_Algoritmos_de_Optimizacion/blob/main/Retos/Algoritmos_R3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Reto 3 (1ª Convocatoria)


He decidido mejorar la implementación del algoritmo genético propuesto para la resolución del TSP mediante la introducción de dos variantes, en primer lugar, en vez de haceer corte en un único punto, he modificado la función realizando un [corte uniforme basándome en una máscara aleatoria](http://www.sc.ehu.es/ccwbayes/docencia/mmcc/docs/t2s.pdf) para cada recombinación de genes y un mecanismo de [selección por torneo probabilístico](http://sabia.tic.udc.es/mgestal/cv/AAGGtutorial/node11.html)

In [ ]:
!pip install requests
import urllib.request

!pip install tsplib95
import tsplib95


In [3]:
import random
import copy

In [6]:
file = "swiss42.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz
problem = tsplib95.load(file)

Nodos = list(problem.get_nodes())

In [7]:
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:] :
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

def distancia(a,b, problem):
  return problem.get_weight(a,b)


def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [14]:
def generar_poblacion(Nodos,N):
  return [crear_solucion(Nodos) for _ in range(N) ]

def Evaluar_Poblacion(poblacion, problem):
  mejor_solucion = []
  mejor_distancia = 10e100
  for p in poblacion:
    distancia_referencia = distancia_total(p, problem)
    if distancia_referencia < mejor_distancia:
      mejor_solucion = p
      mejor_distancia = distancia_referencia
  return mejor_solucion, mejor_distancia


def Cruzar(poblacion,mutacion, problem):
  poblacion_copia = copy.deepcopy(poblacion)

  poblacion_final = copy.deepcopy(poblacion)

  while len(poblacion_copia) > 1:
    padre1,padre2 = random.sample(poblacion_copia   ,2)
    poblacion_copia.remove(padre1)
    poblacion_copia.remove(padre2)
    poblacion_final.extend(Descendencia([padre1,padre2],problem,mutacion))
  return poblacion_final

# Funcion para generar hijos a partir de 2 padres:
# Se ha modificaod la función de 1-punto de corte por una uniforme basándose en una máscara
def Descendencia(padres, problem,mutacion):
  #Se genera una máscara
  random_mask = [random.choice([0, 1]) for _ in range(len(padres[0]))]

  # En mask tenemos una serie de bits. Cada hijo heredará los genes de un padre o de otro en
  # función del valor de dicho bit
  hijo1 = []
  hijo2 = []
  for index, bit in enumerate(random_mask):
    hijo1.append(padres[bit][index])
    hijo2.append(padres[not bit][index])

  # El proceso de Factibilizar es el mismo
  hijo1 =  Factibilizar(hijo1, problem)
  hijo2 =  Factibilizar(hijo2, problem)
  return [hijo1,hijo2,Mutar(hijo1,mutacion),Mutar(hijo2,mutacion)]

def Factibilizar(solucion,problem):
  Nodos = list(problem.get_nodes())
  nodos_desaparecidos = list(set(Nodos) - set(solucion))
  for i in range(len(solucion) ):
    if solucion[i] in solucion[:i]:
      solucion[i] = nodos_desaparecidos.pop(0)
  return solucion


def Mutar(solucion,mutacion):
  if random.random() < mutacion:
    sel1,sel2 = sorted(random.sample(set(solucion)-{solucion[0]},2))
    return solucion[:sel1] + [solucion[sel2]] + solucion[sel1+1:sel2] + [solucion[sel1]]  + solucion[sel2+1:]
  else:
    return solucion[::]

#Funcion de seleccion de la población. Recibe como parametro una poblacion y
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
# Se va a implementar un criterio de selecciónpor torneo, en concreto en su versión probabilística
def Seleccionar(problem,poblacion, N, elitismo):
  # Seleccionamos de forma aleatoria a n individuos (en este caso p=2) y el más apto proocionará el torneo con una probabilidad p = 70%
  n = 2
  p = 0.7
  rand = random.random()
  poblacion_candidata = copy.deepcopy(poblacion)
  poblacion_final = []
  while(len(poblacion_final) < N):
    part1, part2 = random.sample(poblacion_candidata, 2)
    if distancia_total(part1, problem) < distancia_total(part2, problem):
      # part1 es mas apto
      if rand < p:
        poblacion_final.append(part1)
        #Lo eliminamos de los candidatos
        poblacion_candidata.remove(part1)
      else:
        poblacion_final.append(part2)
        poblacion_candidata.remove(part2)
    else:
      # part2 es mas apto (o igual en cuyo caso da igual)
      if rand < p:
        poblacion_final.append(part2)
        poblacion_candidata.remove(part2)
      else:
        poblacion_final.append(part1)
        poblacion_candidata.remove(part1)

  return poblacion_final




In [15]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(problem=problem,N=100,mutacion=.15,elitismo=.1,generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  Nodos = list(problem.get_nodes())
  poblacion = generar_poblacion(Nodos,N)

  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)


  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :

    #Cruce de la poblacion(incluye mutación)
    poblacion = Cruzar(poblacion, mutacion, problem)

    #Seleccionamos la población
    poblacion = Seleccionar(problem,poblacion, N, elitismo)

    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion, problem)

    print("Generacion #", n, "\nLa mejor solución es:" , mejor_solucion, "\ncon distancia " , mejor_distancia, "\n")

    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1

  return mejor_solucion

sol = algoritmo_genetico(problem=problem,N=50,mutacion=.3,elitismo=.40,generaciones=25)

<ipython-input-14-09654c73c826>:57: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sel1,sel2 = sorted(random.sample(set(solucion)-{solucion[0]},2))


Generacion # 0 
La mejor solución es: [0, 15, 32, 27, 14, 35, 17, 24, 34, 31, 19, 3, 41, 1, 33, 37, 36, 39, 7, 26, 2, 21, 4, 25, 6, 28, 38, 20, 5, 23, 8, 30, 22, 29, 40, 9, 10, 11, 16, 13, 12, 18] 
con distancia  4198 

Generacion # 1 
La mejor solución es: [0, 14, 10, 28, 29, 1, 36, 13, 26, 17, 41, 2, 4, 20, 19, 3, 31, 33, 35, 37, 15, 5, 24, 38, 7, 16, 34, 32, 22, 6, 11, 23, 8, 27, 39, 9, 40, 12, 18, 21, 25, 30] 
con distancia  4012 

Generacion # 2 
La mejor solución es: [0, 18, 1, 3, 32, 35, 2, 14, 7, 22, 34, 28, 30, 20, 5, 6, 31, 4, 36, 37, 41, 23, 9, 38, 13, 21, 11, 39, 24, 8, 40, 26, 10, 27, 12, 15, 16, 19, 33, 17, 29, 25] 
con distancia  4140 

Generacion # 3 
La mejor solución es: [0, 18, 1, 3, 32, 35, 2, 14, 7, 22, 34, 28, 30, 20, 5, 6, 31, 4, 36, 37, 41, 23, 9, 38, 13, 21, 11, 39, 24, 8, 40, 26, 10, 27, 12, 15, 16, 19, 33, 17, 29, 25] 
con distancia  4140 

Generacion # 4 
La mejor solución es: [0, 13, 21, 30, 37, 1, 26, 14, 22, 2, 24, 8, 32, 18, 28, 5, 3, 35, 20, 4, 38, 34, 